<a href="https://colab.research.google.com/github/Kuzay3t/Image-Classification/blob/main/Copy_of__Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification Starter Notebook

Welcome to the Image Classification Starter Notebook for our Hacktoberfest competition! 🚀

In this notebook, you'll find a ready-to-use Python script that provides a solid foundation for building an image classifier to classify buildings into three classes: Bungalow, Highrise, and Storey buildings.

## Getting Started

To get started, follow these steps:

1. **Clone the Repository**: Begin by cloning this repository to your local machine.

2. **Organize Your Data**: Ensure that your image data is organized in the `Data` directory, with subdirectories for each class (`Bungalow`, `Highrise`, `Storey-Building`).

3. **Open the Notebook**: Open this notebook in a Jupyter environment.

4. **Follow the Code**: The notebook contains commented code that guides you through the process of setting up the data, building and training the model, and evaluating its performance.

5. **Experiment and Contribute**: Feel free to experiment with different architectures, hyperparameters, or augmentation techniques. If you come up with improvements, consider contributing them back to the project!

## Important Notes

- Ensure that you have TensorFlow and related dependencies installed in your environment.
- If you encounter any issues or have questions, don't hesitate to reach out. We're here to help!

Happy coding, and let's build an amazing image classifier together!


In [2]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from tensorflow.keras.layers import Dropout

In [7]:
!ls /content


 10B21B6B-95F5-4079-8628-6DAE76E348B4.jpeg
 13B889F1-556B-47B3-8FE0-1BD4E1411042.jpeg
 13F5894E-50CE-490E-A037-157FA1FB04ED.jpeg
 20230927_182105.jpeg
 20231004_112620.jpeg
 20231004_115629.jpeg
 20231004_115646.jpeg
 20231004_115650.jpeg
 20231004_120132.jpeg
 20231004_120153.jpeg
 20231004_120200.jpeg
 233C4D1A-D26A-4146-923F-0E7423C25E75.jpeg
 2525339E-981A-41D8-ACF8-090646D92AB4.jpeg
 346B19E7-D0D6-4EFD-BF9E-516BF2EF88DA.jpeg
 37299791-9D8A-4329-8F31-6520F9DF40FE.jpeg
 3EAFA182-ADEC-4D27-9C19-C46FFEACA5A4.jpeg
 46DC1D81-E5CC-4FBF-B88D-DDFB90532C18.jpeg
 53F9FA3C-2DEA-47AB-B69E-8DCCDA120172.jpeg
 59E94C93-1C1E-4D39-BB24-D90087C1E992.jpeg
'5-Bedroom-Duplex-Houses-For-sale-at-All-Nigeria copy.jpeg'
 5-Bedroom-Duplex-Houses-For-sale-at-All-Nigeria.jpeg
 5F4EE75D-71C3-4183-ABB3-1C3A75BCCCBA.jpeg
 6398F29D-B695-4DDB-AFEC-222E92D054E6.jpeg
 682FDE99-703D-49AB-9C8E-6A8BF4DC8295.jpeg
 6AF68D33-53F8-4584-B55F-46862E6DCBE1.jpeg
 7F0E2688-77C8-4EEF-BEF5-63A8A0FAE7B5.jpeg
 81302B3F-5D25-4FED-92

In [10]:
import zipfile
import os

zip_file_path = "/content/your_uploaded_file.zip"
import zipfile
import os

zip_file_path = "/content/your_uploaded_file.zip"
# Extracting
extract_path = "/content/my_folder"
with zipfile.ZipFile("Building Types.zip", "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Verify extraction
print("Extracted files:", os.listdir(extract_path))
# Verify extraction
print("Extracted files:", os.listdir(extract_path))


Extracted files: ['__MACOSX', 'Building Types']
Extracted files: ['__MACOSX', 'Building Types']


# Define Constants

In [11]:

IMAGE_HEIGHT = 400 # The height the image is to be resized to
IMAGE_WIDTH = 300 # The width the image is to be resized to
BATCH_SIZE = 32

# Define paths for data
data_dir = 'data' # Ensure the name of the folder is set to this.
class_names = ['Bungalow', 'Highrise', 'Storey-Building']



# Implement Data Generators and Preprocessing pipeline

**This will create an easy preprocessing pipeline and will help load your data in batches**

In [16]:
# Set up data generators for training and testing
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0,1]
    validation_split=0.2,      # 20% of data will be used for validation
    rotation_range=40,          # Rotate images by up to 40 degrees
    width_shift_range=0.2,     # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2,    # Shift images vertically by up to 20% of the height
    shear_range=0.2,           # Apply shear transformations
    zoom_range=0.2,            # Zoom in or out by up to 20%
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill in missing pixels using the nearest neighbor
)

data_dir = 'my_folder/Building Types'

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)



Found 959 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


**Load InceptionV3 and Fine-Tune the Model**

In [ ]:
#Loading the pre-trained model InceptionV3
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',  # Loading weights pre-trained on ImageNet
    include_top=False,
    input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))  # image dimensions


# Freezing the layers in the base model
base_model.trainable = False

# Adding custom layers on top of the pre-trained model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),  # Pooling layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes for building types
])

# 4. Compiling the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5. Train the model (fine-tuning)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 466s 15s/step - accuracy: 0.5979 - loss: 0.8843 - val_accuracy: 0.7185 - val_loss: 0.5961
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 449s 15s/step - accuracy: 0.7950 - loss: 0.4830 - val_accuracy: 0.7605 - val_loss: 0.5402
Epoch 3/10
17/30 ━━━━━━━━━━━━━━━━━━━━ 2:36 12s/step - accuracy: 0.8110 - loss: 0.4504

# Model architecture

In [14]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add a dropout layer with a rate of 0.5
    tf.keras.layers.Dense(3, activation='softmax')  # 3 classes for building types
])

# Compile the model
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])



/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Train the model

In [ ]:

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
# This part of the code allows you to train the model. You can decide to tweak the number of epochs by experimenting with different values.
# Rule of thumb: Avoid using too large epochs, to avoid overfitting.



# Evaluate the model

In [ ]:

test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print(f'Test accuracy: {test_acc*100:.2f}%')


# Save the Model

In [ ]:
path_to_save_model = 'path_to_save_model'

# Save the entire model (including architecture, weights, and optimizer state)
model.save(path_to_save_model)

# If you only want to save the model architecture and weights (without optimizer state)
# Uncomment the following line and comment out the previous 'model.save()' line
# model.save(path_to_save_model, save_format='tf')

## **Changes made by Bagai Glory**


1.   I included Data Augumentation parameters withing the ImageDataGenrator
2.   Fine tuning the model using InceptionV3
3. Included the dropout function to manage overfitting in the data.
4. Implement a learning rate schedule or use adaptive learning rate methods like Adam.

